In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
import bentoml
NAME = 'linear_sleep_clipper'

# load data & train model(not used, just placeholder)

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

iris = datasets.load_iris()
x = iris.data[:, 2:]
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

    
# add parameters for tuning
num_estimators = 100

# train the model
model = RandomForestRegressor(n_estimators=num_estimators)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('predictions: ', predictions)

# log model performance 
mse = mean_squared_error(y_test, predictions)
print("  mse: %f" % mse)

predictions:  [2.   1.   0.   1.   1.61 0.   1.24 1.   0.   1.   1.98 1.   0.   2.
 0.   1.88 2.   2.   0.   0.   1.   2.   1.   1.36 1.53 1.88 1.   1.
 2.   2.  ]
  mse: 0.086313


In [3]:
%%writefile {NAME}.py

import bentoml
import time
from bentoml.artifact import SklearnModelArtifact
from bentoml.handlers import ClipperFloatsHandler


@bentoml.env()
@bentoml.artifacts([SklearnModelArtifact('model')])
class BentoSvc(bentoml.BentoService):

    @bentoml.api(ClipperFloatsHandler)
    def predict_clipper(self, inputs):
        a, b = inputs[0]
        x = inputs.shape[0]
        time.sleep(a * x + b)
        return inputs

Overwriting linear_sleep_clipper.py


In [4]:
from linear_sleep_clipper import BentoSvc

bento_svc = BentoSvc()
bento_svc.pack("model", model)
saved_path = bento_svc.save()

[2020-05-13 14:19:20,949] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-13 14:19:21,058] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'e2e_tests'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'benchmark'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check
creating BentoML-0.4.9+224.g4061229.dirty
creating BentoML-0.4.9+224.g4061229.dirty/BentoML.egg-info
creating BentoML-0.4.9+224.g4061229.dirty/bentoml
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/artifact
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/bundler
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/cli
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/clipper
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/configuration
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/configuration/__pycache__
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/deployment
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/deployment/aws_lambda
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/deployment/sagemaker
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/handlers
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/marshal
creating BentoML-0.4.9+224.g4061229.dirty/bentoml/migrations
creating BentoML-0.4.9+224.g406

# Build & Run Bento Service in Docker

In [5]:
from clipper_admin import ClipperConnection, DockerContainerManager
from bentoml.clipper import deploy_bentoml
cl = ClipperConnection(DockerContainerManager())
try:
    cl.start_clipper(cache_size=1)
except:
    cl.connect()

APP_NAME = saved_path.split('/')[-1].lower()
cl.register_application(APP_NAME, 'floats', 'default_pred', 10 * 1000* 1000)  # 10s SLO(max_latency for bentoml)

20-05-13:14:19:36 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-05-13:14:19:39 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmpoy2lyj01.yml
20-05-13:14:19:40 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
20-05-13:14:19:40 INFO     [clipper_admin.py:236] [default-cluster] Application 20200513141921_e41e2d was successfully registered


In [6]:
clipper_model_name, clipper_model_version = deploy_bentoml(
    cl, saved_path, 'predict_clipper'
)

[2020-05-13 14:19:45,810] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-13 14:19:45,821] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.4.9,  but loading from BentoML version 0.4.9+224.g4061229.dirty
[2020-05-13 14:19:46,480] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image

20-05-13:14:20:38 INFO     [docker_container_manager.py:409] [default-cluster] Found 0 replicas for bentosvc-predict-clipper:20200513141921-e41e2d. Adding 1
20-05-13:14:20:39 INFO     [clipper_admin.py:724] [default-cluster] Successfully registered model bentosvc-predict-clipper:20200513141921-e41e2d
20-05-13:14:20:39 INFO     [clipper_admin.py:642] [default-cluster] Done deploying model bentosvc-predict-clipper:20200513141921-e41e2d.


[2020-05-13 14:20:39,030] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.4.9. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-05-13 14:20:39,047] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.4.9,  but loading from BentoML version 0.4.9+224.g4061229.dirty


In [7]:
cl.set_num_replicas(clipper_model_name, 1)
cl.link_model_to_app(APP_NAME, clipper_model_name)
addr = cl.get_query_addr()
server_url = f"http://{addr}/{APP_NAME}/predict"
server_url

20-05-13:14:20:39 INFO     [clipper_admin.py:303] [default-cluster] Model bentosvc-predict-clipper is now linked to application 20200513141921_e41e2d


'http://localhost:1337/20200513141921_e41e2d/predict'

# Test with requests

In [15]:
server_url = 'http://localhost:1337/20200513141921_e41e2d/predict'
import json
import random
import requests


headers = {"content-type": "application/json"}
data = json.dumps(
       {"input": [0.0, 2.0 + random.random()/ 10000]}
)

json_response = requests.post(server_url, data=data, headers=headers)
print(json_response)
print(json_response.text)

<Response [200]>
{"query_id":112443,"output":"[0.        2.0000994]","default":false}


# Benchmark

In [16]:
import random
import ujson as json

def get_request_producer(A, B):

    headers = {"content-type": "application/json"}

    def _gen_data():
        _A = A + random.random() / 100000.0
        _B = B + random.random() / 10000.0
        data = json.dumps(
               {"input": [_A, _B]}  # random offset to avoid cache
        )
        return server_url, "POST", headers, data

    return _gen_data

# When it reached the limit of clipper, clipper will response with the default value
def verify_clipper_response(status, msg):
    if status // 100 == 2 and "default_pred" not in msg:
        return True
    return False

get_request_producer(A=0.1, B=0.5)()

('http://localhost:1337/20200513141921_e41e2d/predict',
 'POST',
 {'content-type': 'application/json'},
 '{"input":[0.1000035958,0.5000345512]}')

In [27]:
A = 0.01
B = 0

# benchmark under a proper amount of users (five times of the theoretical capacity)
proper_user_num = max(int(5 / A), 1)
print("test user amount: ", proper_user_num)

from bentoml.utils.benchmark import BenchmarkClient
b = BenchmarkClient(get_request_producer(A=A, B=B), lambda: 1, timeout=11,
                    verify_response=verify_clipper_response)
b.start_session(60, proper_user_num, proper_user_num)

test user amount:  500
======= Session started! =======

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘══════════╧═════════╧══════════╧═════════════════╧═══════════════════╛

╒══════════╤═════════╤══════════╤═════════════════╤═══════════════════╕
│ Result   │   Total │   Reqs/s │   Resp Time Avg │ Client Health %   │
╞══════════╪═════════╪══════════╪═════════════════╪═══════════════════╡
│ succ     │       0 │        0 │             nan │ 100.0             │
├──────────┼─────────┼──────────┼─────────────────┼───────────────────┤
│ fail     │       0 │        0 │             nan │                   │
╘═════

In [26]:
def log_result(b, name, file_name):
    self = b.stat

    result = []
    result.append(name)
    
    result.append(self.success)
    result.append(self.success / max(self.sess_time, 1))
    result.append(sum(self.succ_times) / max(self.success, 1))

    result.append(self.fail)
    result.append(self.fail / max(self.sess_time, 1))
    result.append(sum(self.exec_times) / max(self.fail, 1))
    log_str = ','.join(str(r) for r in result)
    with open(file_name, "a") as lf:
        lf.write(log_str)
        lf.write('\n')


log_result(b, f'{NAME}_{A}_{B}', 'benchmark_result.csv')